In [5]:
import numpy as np
import pandas as pd

%matplotlib inline 
import matplotlib.pyplot as plt

import datetime
from datetime import datetime, date, time

import time

from sklearn import svm
from sklearn import datasets

In [6]:


scenarios = ['Entrainement' ,'Facile', 'Difficile']
 
    
HRV_clean = pd.DataFrame()
HRV = pd.DataFrame()
Diam_Right_clean = pd.DataFrame()
Diam_Left_clean = pd.DataFrame()
total = pd.DataFrame()

    
for scenario in scenarios:
    filename_HRV = 'S5/Sujet1/Sujet1_'+ scenario +'_RecCSVFile/HRV.csv'
    filename_pupleft = 'S5/Sujet1/Sujet1_'+ scenario +'_RecCSVFile/Eye1.csv'
    filename_pupright = 'S5/Sujet1/Sujet1_'+ scenario +'_RecCSVFile/Eye2.csv'

    files = [filename_HRV, filename_pupleft, filename_pupright]

    for filename in files: 

        file = pd.read_csv(filename, sep = ';')
        

        
        file['Timestamp'] = file[file.columns[0]] 
         
        file['scenario'] = scenario
        
        if filename == filename_HRV: 
            V = []
            file['HRV'] = file[file.columns[1]] 
            file = file.drop(file.columns[1],1)
            file.index = np.arange(1,len(file)+1)
            HRV = pd.concat([HRV, file])

            for i in range(1,len(file), 50): 
                V.append(i)
            for i in file.index: 
                if i not in V : 
                    file = file.drop(i,0)


            mask = file['HRV']==-1
            file = file[~mask]


            mean = file['HRV'].mean()
            standard_dev = file['HRV'].std()

            L = [mean, standard_dev, scenario]
           
            
            HRV_clean = pd.concat([HRV_clean, file])



        elif filename == filename_pupleft : 
            file['Diam Occ Left'] = file[file.columns[1]] 
            file = file.drop(file.columns[1],1)
            mask = file['Diam Occ Left'] ==0
            file = file[~mask] 
            mean = file['Diam Occ Left'].mean()
            standard_dev = file['Diam Occ Left'].std()

            M = [mean, standard_dev, scenario]

            
            
            Diam_Left_clean = pd.concat([Diam_Left_clean, file])


        else : 
            
            file['Diam Occ Right'] = file[file.columns[1]] 
            file = file.drop(file.columns[1],1)
            mask = file['Diam Occ Right'] ==0
            file = file[~mask] 
            mean = file['Diam Occ Right'].mean()
            standard_dev = file['Diam Occ Right'].std()

            N = [mean, standard_dev, scenario]
            Diam_Right_clean = pd.concat([Diam_Right_clean, file])


    final = pd.DataFrame([L, M, N],columns = ['Mean', 'Standard Deviation', 'Scenario'] , index = ['HRV', 'Diam Occ Left', 'Diam Occ Right'])
    total = pd.concat([total, final])


In [7]:
HRV_clean['HRV centré'] = (HRV_clean['HRV'] -final['Mean']['HRV'])/final['Standard Deviation']['HRV']
Diam_Right_clean['Diam Occ Right centré'] = (Diam_Right_clean['Diam Occ Right'] -final['Mean']['Diam Occ Right'])/final['Standard Deviation']['Diam Occ Right']
Diam_Left_clean['Diam Occ Left centré'] = (Diam_Left_clean['Diam Occ Left'] -final['Mean']['Diam Occ Left'])/final['Standard Deviation']['Diam Occ Left']



In [8]:
HRV_new = HRV
HRV_clean_final = HRV_clean
PupRight = Diam_Left_clean
PupLeft = Diam_Right_clean


In [9]:

V = HRV_new.columns
J = HRV_clean_final.columns
S = PupRight.columns
T = PupLeft.columns

for i in range(3):
    HRV_new = HRV_new.drop(V[i], 1)
    HRV_clean_final = HRV_clean_final.drop(J[i], 1)
    PupRight = PupRight.drop(S[i], 1)
    PupLeft = PupLeft.drop(T[i], 1)
    
        



In [17]:
HRV_data = HRV_clean_final['HRV']
Time = HRV_clean_final['Timestamp'].values